#Mount drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive"

'Colab Notebooks'       down_100set.npy         Ecg_1d_srcnn.h5
 CPSC_500_6sec	        down_125set.npy         gt_set.npy
 cpsc_classlabels.npy   down_20set.npy	        lstm.h5
 CPSC_downsample100Hz   down250_set.npy         Reflections_medicomp
 CPSC_downsample125Hz   down_25set.npy	        TrainingSet1
 CPSC_downsample20Hz    down_50set.npy	        TrainingSet2
 CPSC_downsample250Hz   ECG_1DCNN_30.h5         TrainingSet3
 CPSC_downsample25Hz    ECG_1DCNN_5classes.h5   unet1d_new.h5
 CPSC_downsample50Hz    Ecg_1d_srcnn_100Hz.h5


#CPSC Data: Labels analysis

In [ ]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt

set1_path = "/content/drive/My Drive/TrainingSet1/"
set2_path = "/content/drive/My Drive/TrainingSet2/"
set3_path = "/content/drive/My Drive/TrainingSet3/"
reference_path = "/content/drive/My Drive/TrainingSet3/REFERENCE.csv"

In [ ]:
import pandas as pd

refer = pd.read_csv(reference_path)
refer

In [ ]:
print(refer.Recording[0],'\n',refer.First_label[0])

A0001 
 5


In [ ]:
#1st label is present in all, here we are ust checking how many have 2nd and 3rd labels too
records_2ndlabelOnly = 0
records_3rdlabelOnly = 0
records_2ndAnd3rd = 0

for i in range(len(refer)):
  if np.isnan(refer.Second_label[i]) != True and np.isnan(refer.Third_label[i]) != True:
    records_2ndAnd3rd += 1
  elif (np.isnan(refer.Second_label[i]) != True and np.isnan(refer.Third_label[i])):
    records_2ndlabelOnly += 1
  elif np.isnan(refer.Second_label[i]) and np.isnan(refer.Third_label[i]) != True:
    records_3rdlabelOnly += 1


print(records_2ndlabelOnly)
print(records_3rdlabelOnly)
print(records_2ndAnd3rd)

471
0
6


#Segregation according to arrhythmia class-methods

In [ ]:
arrhythmia_class1 = []
arrhythmia_class2 = []
arrhythmia_class3 = []
arrhythmia_class4 = []
arrhythmia_class5 = []
arrhythmia_class6 = []
arrhythmia_class7 = []
arrhythmia_class8 = []
arrhythmia_class9 = []

In [ ]:
def getTarget(recordname):
  for i in range(len(refer.Recording)):
    if refer.Recording[i] == str(recordname) :
      return refer.First_label[i] #Checking only by first label

def checkArrhythmiaClassndSegregate(arhythmia_class,ecg_record_tuple):
  if arhythmia_class == 1:
    arrhythmia_class1.append(ecg_record_tuple)
  elif arhythmia_class == 2:
    arrhythmia_class2.append(ecg_record_tuple)
  elif arhythmia_class == 3:
    arrhythmia_class3.append(ecg_record_tuple)
  elif arhythmia_class == 4:
    arrhythmia_class4.append(ecg_record_tuple)
  elif arhythmia_class == 5:
    arrhythmia_class5.append(ecg_record_tuple)
  elif arhythmia_class == 6:
    arrhythmia_class6.append(ecg_record_tuple)
  elif arhythmia_class == 7:
    arrhythmia_class7.append(ecg_record_tuple)
  elif arhythmia_class == 8:
    arrhythmia_class8.append(ecg_record_tuple)
  elif arhythmia_class == 9:
    arrhythmia_class9.append(ecg_record_tuple)

#Downsampling and segregation

In [ ]:
from scipy.signal import resample_poly

def getDownsampledEcg(required_fs,ecg_12leads):
  original_fs = 500
  
  if required_fs == 500:
    new_dim = ecg_12leads[0].shape[0]
    #print(new_dim,"500 required!!")
    down_ecg_12leads = np.zeros((12,new_dim))
    for i in range(12):
      down_ecg_12leads[i] = ecg_12leads[i]
  else:
    temp = resample_poly(ecg_12leads[0],up=required_fs,down=original_fs)
    new_dim = temp.shape[0] 
    #print(new_dim,"500 not required")
    down_ecg_12leads = np.zeros((12,new_dim))
    for i in range(12):
      down_ecg_12leads[i] = resample_poly(ecg_12leads[i],up=required_fs,down=original_fs)

  return down_ecg_12leads

In [ ]:
import os
import scipy.io

list_files_set1 = os.listdir(set1_path)
list_files_set2 = os.listdir(set2_path)
list_files_set3 = os.listdir(set3_path)

In [ ]:
fs_target = 500 #change the target fs here
dimension_after_downsample = []

for item in list_files_set1:
  data_cpsc_sample1 = scipy.io.loadmat(set1_path+item)
  #print(set1_path+item)
  rec_name1 = item.split('.')[0]
  ecg_cpsc_arr1 = data_cpsc_sample1['ECG'][0,0][2]
  down_sample_ecg1 = getDownsampledEcg(fs_target,ecg_cpsc_arr1)
  dimension_after_downsample.append(down_sample_ecg1.shape[1])
  checkArrhythmiaClassndSegregate(getTarget(rec_name1),[rec_name1,down_sample_ecg1])

for item in list_files_set2:
  data_cpsc_sample2 = scipy.io.loadmat(set2_path+item)
  #print(set2_path+item)
  rec_name2 = item.split('.')[0]
  ecg_cpsc_arr2 = data_cpsc_sample2['ECG'][0,0][2]
  down_sample_ecg2 = getDownsampledEcg(fs_target,ecg_cpsc_arr2)
  dimension_after_downsample.append(down_sample_ecg2.shape[1])
  checkArrhythmiaClassndSegregate(getTarget(rec_name2),[rec_name2,down_sample_ecg2])

for item in list_files_set3:
  if item != "REFERENCE.csv":
    data_cpsc_sample3 = scipy.io.loadmat(set3_path+item)
    print(set3_path+item)
    rec_name3 = item.split('.')[0]
    ecg_cpsc_arr3 = data_cpsc_sample3['ECG'][0,0][2]
    down_sample_ecg3 = getDownsampledEcg(fs_target,ecg_cpsc_arr3)
    dimension_after_downsample.append(down_sample_ecg3.shape[1])
    checkArrhythmiaClassndSegregate(getTarget(rec_name3),[rec_name3,down_sample_ecg3])


In [ ]:
print(len(dimension_after_downsample))
trunc_length = min(dimension_after_downsample)
print(trunc_length)

6877
3000


In [ ]:
print(len(arrhythmia_class1))
print(len(arrhythmia_class2))
print(len(arrhythmia_class3))
print(len(arrhythmia_class4))
print(len(arrhythmia_class5))
print(len(arrhythmia_class6))
print(len(arrhythmia_class7))
print(len(arrhythmia_class8))
print(len(arrhythmia_class9))

918
1098
704
207
1695
574
653
826
202


In [ ]:
#create labels file for all leads data:
import numpy as np

normal_labels = np.ones(len(arrhythmia_class1))  ###1
af_labels = 2 * np.ones(len(arrhythmia_class2)) ####2
iavb_labels = 3 * np.ones(len(arrhythmia_class3)) ###3
lbbb_labels = 4 * np.ones(len(arrhythmia_class4)) ###4
rbbb_labels = 5 * np.ones(len(arrhythmia_class5)) ###5
pac_labels = 6 * np.ones(len(arrhythmia_class6))
pvc_labels = 7 * np.ones(len(arrhythmia_class7))
std_labels = 8 * np.ones(len(arrhythmia_class8))
ste_labels = 9 * np.ones(len(arrhythmia_class9))

cpsc_leadwise_labels = np.concatenate((normal_labels,af_labels,iavb_labels,lbbb_labels,rbbb_labels,pac_labels,pvc_labels,std_labels,ste_labels))

In [ ]:
np.shape(cpsc_leadwise_labels)

(6877,)

In [ ]:
np.save('cpsc_classlabels_leadwise',cpsc_leadwise_labels)

In [ ]:
import shutil

shutil.move('cpsc_classlabels_leadwise.npy', "/content/drive/My Drive/")

'/content/drive/My Drive/cpsc_classlabels_leadwise.npy'

#Converting to dataframe and moving to drive

In [ ]:
lead_0_gtdata = []
lead_1_gtdata = []
lead_2_gtdata = []
lead_3_gtdata = []
lead_4_gtdata = []
lead_5_gtdata = []
lead_6_gtdata = []
lead_7_gtdata = []
lead_8_gtdata = []
lead_9_gtdata = []
lead_10_gtdata = []
lead_11_gtdata = []

In [ ]:
from scipy.signal import resample_poly

def separateChannelWiseEcg(trunc_ecg_all_leads,channel_ind):
  ch_data = trunc_ecg_all_leads[:,channel_ind,:]
  if channel_ind == 0:
    lead_0_gtdata.append(ch_data)
  elif channel_ind == 1:
    lead_1_gtdata.append(ch_data)
  elif channel_ind == 2:
    lead_2_gtdata.append(ch_data)
  elif channel_ind == 3:
    lead_3_gtdata.append(ch_data)
  elif channel_ind == 4:
    lead_4_gtdata.append(ch_data)
  elif channel_ind == 5:
    lead_5_gtdata.append(ch_data)
  elif channel_ind == 6:
    lead_6_gtdata.append(ch_data)
  elif channel_ind == 7:
    lead_7_gtdata.append(ch_data)
  elif channel_ind == 8:
    lead_8_gtdata.append(ch_data)
  elif channel_ind == 9:
    lead_9_gtdata.append(ch_data)
  elif channel_ind == 10:
    lead_10_gtdata.append(ch_data)
  elif channel_ind == 11:
    lead_11_gtdata.append(ch_data)

  

def getChannelDataForArrhythmiaAndStore(channel_index,arh_class_data,arh_class_index):
  recordsarr = np.array(arh_class_data)[:,0]
  arrhythmia_class_tracings = np.array(arh_class_data)[:,1]
  arh_data_mat = np.zeros((len(arh_class_data),12,trunc_length),dtype=np.float32)
  for i in range(len(arrhythmia_class_tracings)):
    arh_data_mat[i] = arrhythmia_class_tracings[i][:,0:trunc_length]
  separateChannelWiseEcg(arh_data_mat,channel_index)

  return recordsarr

In [ ]:
from scipy.signal import resample_poly

def getChannelDataForParticularArrhythmia(channel_index,arh_class_data,arh_class_index):
  recordsarr = np.array(arh_class_data)[:,0]
  arrhythmia_class_tracings = np.array(arh_class_data)[:,1]
  arh_data_mat = np.zeros((len(arh_class_data),12,trunc_length),dtype=np.float32)
  for i in range(len(arrhythmia_class_tracings)):
    arh_data_mat[i] = arrhythmia_class_tracings[i][:,0:trunc_length]
  ch_data = arh_data_mat[:,channel_index,:]
  temp = {recordsarr[i]: ch_data[i] for i in range(len(recordsarr))}
  filename = "version_"+str(fs_target)+"Hz_class_"+str(arh_class_index)+"_lead_"+str(channel_index)+"_chunk_"+str(trunc_length)+".parquet"
  final_df = pd.DataFrame(data = temp)
  final_df.to_parquet(filename)

In [ ]:
recordsarr_all_channel_indices = []

In [ ]:
#to get leadwise data
segregated_arh_listarray = [arrhythmia_class1,arrhythmia_class2,arrhythmia_class3,
                            arrhythmia_class4,arrhythmia_class5,arrhythmia_class6,
                            arrhythmia_class7,arrhythmia_class8,arrhythmia_class9]

for i in range(9):
  for j in range(12):
    recordsarr_all_channel_indices.append(getChannelDataForArrhythmiaAndStore(j,segregated_arh_listarray[i],i+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
segregated_arh_listarray = [arrhythmia_class1,arrhythmia_class2,arrhythmia_class3,
                            arrhythmia_class4,arrhythmia_class5,arrhythmia_class6,
                            arrhythmia_class7,arrhythmia_class8,arrhythmia_class9]

for i in range(9):
  for j in range(12):
    getChannelDataForParticularArrhythmia(j,segregated_arh_listarray[i],i+1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [ ]:
#move from temp storage to your drive
import shutil

for i in range(1,10):
  for j in range(12):
    filepath = '/content/version_'+str(fs_target)+'Hz_class_'+str(i)+'_lead_'+str(j)+'_chunk_'+str(trunc_length)+'.parquet'

    #print(filepath)
    shutil.move(filepath, "/content/drive/My Drive/CPSC_downsample125Hz")

#Prepare lead-wise balanced data

In [ ]:
import numpy as np

In [ ]:
leadwise_balanced_ecg = []
leadwise_balanced_labels = []

In [ ]:
def getLeadwiseBalancedData(gt_data,gt_labels):
  cpsc_labels = gt_labels
  gt_ecg = gt_data

  ecg_normal = []
  ecg_af = []
  ecg_iavb = []
  ecg_lbbb = []
  ecg_rbbb = []
  ecg_pac = []
  ecg_pvc = []
  ecg_std = []
  ecg_ste = []

  normal_labels = []
  af_labels = []
  iavb_labels = []
  lbbb_labels = []
  rbbb_labels = []
  pac_labels = []
  pvc_labels = []
  std_labels = []
  ste_labels = []
  
  for i in cpsc_labels:
    if i == 1:
      ecg_normal.append(gt_ecg[i])
      normal_labels.append(i)
    elif i == 2:
      ecg_af.append(gt_ecg[i])
      af_labels.append(i)
    elif i == 3:
      ecg_iavb.append(gt_ecg[i])
      iavb_labels.append(i)
    elif i == 4:
      ecg_lbbb.append(gt_ecg[i])
      lbbb_labels.append(i)
    elif i == 5:
      ecg_rbbb.append(gt_ecg[i])
      rbbb_labels.append(i)
    elif i == 6:
      ecg_pac.append(gt_ecg[i])
      pac_labels.append(i)
    elif i == 7:
      ecg_pvc.append(gt_ecg[i])
      pvc_labels.append(i)
    elif i == 8:
      ecg_std.append(gt_ecg[i])
      std_labels.append(i)
    elif i == 9:
      ecg_ste.append(gt_ecg[i])
      ste_labels.append(i)
      
  min_readings_classwise = min(len(ecg_normal),len(ecg_af),len(ecg_iavb),len(ecg_lbbb),len(ecg_rbbb),len(ecg_pac),len(ecg_pvc),len(ecg_std),len(ecg_ste))
  ecg_normal = ecg_normal[:min_readings_classwise]
  ecg_af = ecg_af[:min_readings_classwise]
  ecg_iavb = ecg_iavb[:min_readings_classwise]
  ecg_lbbb = ecg_lbbb[:min_readings_classwise]
  ecg_rbbb = ecg_rbbb[:min_readings_classwise]
  ecg_pac = ecg_pac[:min_readings_classwise]
  ecg_pvc = ecg_pvc[:min_readings_classwise]
  ecg_std = ecg_std[:min_readings_classwise]
  ecg_ste = ecg_ste[:min_readings_classwise]

  normal_labels = normal_labels[:min_readings_classwise]
  af_labels = af_labels[:min_readings_classwise]
  iavb_labels = iavb_labels[:min_readings_classwise]
  lbbb_labels = lbbb_labels[:min_readings_classwise]
  rbbb_labels = rbbb_labels[:min_readings_classwise]
  pac_labels = pac_labels[:min_readings_classwise]
  pvc_labels = pvc_labels[:min_readings_classwise]
  std_labels = std_labels[:min_readings_classwise]
  ste_labels = ste_labels[:min_readings_classwise]

  lead_ecg_set = np.concatenate((ecg_normal,ecg_af,ecg_iavb,ecg_lbbb,ecg_rbbb,ecg_pac,ecg_pvc,ecg_std,ecg_ste),axis=0)
  lead_ecg_labels = np.concatenate((normal_labels,af_labels,iavb_labels,lbbb_labels,rbbb_labels,pac_labels,pvc_labels,std_labels,ste_labels))
  
  leadwise_balanced_ecg.append(lead_ecg_set)
  leadwise_balanced_labels.append(lead_ecg_labels)

In [ ]:
gt_main_path = "/content/drive/My Drive/Leadwise_data/gtdata_lead_"
gt_labels_mainpath = "/content/drive/My Drive/Leadwise_data/labels_lead"
for i in range(12):
  lead_gt = np.load(gt_main_path + str(i) + ".npy")
  lead_labels = np.load(gt_labels_mainpath + str(i) + ".npy")
  getLeadwiseBalancedData(lead_gt,lead_labels)

In [ ]:
gt_cpsc_balanced = np.concatenate(leadwise_balanced_ecg,axis = 0)
gt_labels_balanced = np.concatenate(leadwise_balanced_labels)

In [ ]:
print(gt_cpsc_balanced.shape)
print(gt_labels_balanced.shape)

(21816, 3000)
(21816,)


In [ ]:
np.save("cpsc_leadclass_balanced",gt_cpsc_balanced)
np.save("cpsc_leadclass_balanced_labels",gt_labels_balanced)

In [ ]:
import shutil

shutil.move('cpsc_leadclass_balanced.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('cpsc_leadclass_balanced_labels.npy', "/content/drive/My Drive/Leadwise_data/")

'/content/drive/My Drive/Leadwise_data/cpsc_leadclass_balanced_labels.npy'

#Prepare dataset


This section is just a longer aternative to the previous setion that needs to be executed only once.

In [ ]:
import os
import pandas as pd
import numpy as np

gt_path = "/content/drive/My Drive/CPSC_500_6sec/"
down_250_path = "/content/drive/My Drive/CPSC_downsample250Hz/"
down_125_path = "/content/drive/My Drive/CPSC_downsample125Hz/"
down_100_path = "/content/drive/My Drive/CPSC_downsample100Hz/"
down_50_path = "/content/drive/My Drive/CPSC_downsample50Hz/"
down_25_path = "/content/drive/My Drive/CPSC_downsample25Hz/"
down_20_path = "/content/drive/My Drive/CPSC_downsample20Hz/"

In [ ]:
import numpy as np

complete_data_dir_gt = os.listdir(gt_path)
complete_data_dir_250 = os.listdir(down_250_path)
#complete_data_dir_125 = os.listdir(down_125_path)
#complete_data_dir_100 = os.listdir(down_100_path)
#complete_data_dir_50 = os.listdir(down_50_path)
#complete_data_dir_25 = os.listdir(down_25_path)
#complete_data_dir_20 = os.listdir(down_20_path)


gt_set = []
down_250set = []
#down_125set = []
#down_100set = []
#down_50set = []
#down_25set = []
#down_20set = []

arrhythmia_class_labels = []
for item in complete_data_dir_gt:
  gt_500 = pd.read_parquet(gt_path+item)
  orig_fname = item.split('_')
  index_class = orig_fname.index('class')
  orig_class = orig_fname[index_class+1]
  index_lead = orig_fname.index('lead')
  orig_lead = orig_fname[index_lead+1]
  for entry in np.array(gt_500.T):
    gt_set.append(entry)

  for down_item in complete_data_dir_250:
    fname = down_item.split('_')
    i_class = fname.index('class')
    o_class = fname[i_class+1]
    i_lead = fname.index('lead')
    o_lead = fname[i_lead+1]
    if o_class == orig_class and o_lead == orig_lead:
      down_250 = pd.read_parquet(down_250_path+down_item)
      for entry_down in np.array(down_250.T):
          down_250set.append(entry_down)
          arrhythmia_class_labels.append(int(o_class))

In [ ]:
np.save('down_20set',np.array(down_125set))

In [ ]:
import shutil

shutil.move('down_20set.npy', "/content/drive/My Drive/")

'/content/drive/My Drive/down_20set.npy'

In [ ]:
np.save('gt_set',np.array(gt_set))
np.save('cpsc_classlabels',np.array(arrhythmia_class_labels))
np.save('down250_set',np.array(down_250set))

In [ ]:
#move from temp storage to your drive
import shutil

shutil.move('gt_set.npy', "/content/drive/My Drive/")
shutil.move('cpsc_classlabels.npy', "/content/drive/My Drive/")
shutil.move('down250_set.npy', "/content/drive/My Drive/")

'/content/drive/My Drive/down250_set.npy'

##Segregate lead-wise data

In [ ]:
lead_0_gtdata = []
lead_1_gtdata = []
lead_2_gtdata = []
lead_3_gtdata = []
lead_4_gtdata = []
lead_5_gtdata = []
lead_6_gtdata = []
lead_7_gtdata = []
lead_8_gtdata = []
lead_9_gtdata = []
lead_10_gtdata = []
lead_11_gtdata = []

In [ ]:
lead_0_class_labels = []
lead_1_class_labels = []
lead_2_class_labels = []
lead_3_class_labels = []
lead_4_class_labels = []
lead_5_class_labels = []
lead_6_class_labels = []
lead_7_class_labels = []
lead_8_class_labels = []
lead_9_class_labels = []
lead_10_class_labels = []
lead_11_class_labels = []

In [ ]:
import numpy as np


def separateChannelWiseEcg(trunc_ecg_all_leads,channel_ind,label_class):
  ch_data = np.array(trunc_ecg_all_leads)
  count = ch_data.shape[0]

  lead = int(channel_ind)
  if lead == 0:
    lead_0_gtdata.append(ch_data)
    lead_0_class_labels.append(label_class*np.ones(count))
  elif lead == 1:
    lead_1_gtdata.append(ch_data)
    lead_1_class_labels.append(label_class*np.ones(count))
  elif lead == 2:
    lead_2_gtdata.append(ch_data)
    lead_2_class_labels.append(label_class*np.ones(count))
  elif lead == 3:
    lead_3_gtdata.append(ch_data)
    lead_3_class_labels.append(label_class*np.ones(count))
  elif lead == 4:
    lead_4_gtdata.append(ch_data)
    lead_4_class_labels.append(label_class*np.ones(count))
  elif lead == 5:
    lead_5_gtdata.append(ch_data)
    lead_5_class_labels.append(label_class*np.ones(count))
  elif lead == 6:
    lead_6_gtdata.append(ch_data)
    lead_6_class_labels.append(label_class*np.ones(count))
  elif lead == 7:
    lead_7_gtdata.append(ch_data)
    lead_7_class_labels.append(label_class*np.ones(count))
  elif lead == 8:
    lead_8_gtdata.append(ch_data)
    lead_8_class_labels.append(label_class*np.ones(count))
  elif lead == 9:
    lead_9_gtdata.append(ch_data)
    lead_9_class_labels.append(label_class*np.ones(count))
  elif lead == 10:
    lead_10_gtdata.append(ch_data)
    lead_10_class_labels.append(label_class*np.ones(count))
  elif lead == 11:
    lead_11_gtdata.append(ch_data)
    lead_11_class_labels.append(label_class*np.ones(count))

In [ ]:
import numpy as np

complete_data_dir_gt = os.listdir(gt_path)

arrhythmia_class_labels = []
for item in complete_data_dir_gt:
  gt_500 = pd.read_parquet(gt_path+item)
  orig_fname = item.split('_')
  index_class = orig_fname.index('class')
  orig_class = orig_fname[index_class+1]
  index_lead = orig_fname.index('lead')
  orig_lead = orig_fname[index_lead+1]
  separateChannelWiseEcg(gt_500.T,orig_lead,int(orig_class))

In [ ]:
temp = np.concatenate(lead_11_class_labels).astype(int)
len(temp)

6877

In [ ]:
gtdata_lead_0 = np.concatenate(lead_0_gtdata,axis = 0)
gtdata_lead_1 = np.concatenate(lead_1_gtdata,axis = 0)
gtdata_lead_2 = np.concatenate(lead_2_gtdata,axis = 0)
gtdata_lead_3 = np.concatenate(lead_3_gtdata,axis = 0)
gtdata_lead_4 = np.concatenate(lead_4_gtdata,axis = 0)
gtdata_lead_5 = np.concatenate(lead_5_gtdata,axis = 0)
gtdata_lead_6 = np.concatenate(lead_6_gtdata,axis = 0)
gtdata_lead_7 = np.concatenate(lead_7_gtdata,axis = 0)
gtdata_lead_8 = np.concatenate(lead_8_gtdata,axis = 0)
gtdata_lead_9 = np.concatenate(lead_9_gtdata,axis = 0)
gtdata_lead_10 = np.concatenate(lead_10_gtdata,axis = 0)
gtdata_lead_11 = np.concatenate(lead_11_gtdata,axis = 0)

In [ ]:
np.save('gtdata_lead_0',gtdata_lead_0)
np.save('gtdata_lead_1',gtdata_lead_1)
np.save('gtdata_lead_2',gtdata_lead_2)
np.save('gtdata_lead_3',gtdata_lead_3)
np.save('gtdata_lead_4',gtdata_lead_4)
np.save('gtdata_lead_5',gtdata_lead_5)
np.save('gtdata_lead_6',gtdata_lead_6)
np.save('gtdata_lead_7',gtdata_lead_7)
np.save('gtdata_lead_8',gtdata_lead_8)
np.save('gtdata_lead_9',gtdata_lead_9)
np.save('gtdata_lead_10',gtdata_lead_10)
np.save('gtdata_lead_11',gtdata_lead_11)

In [ ]:
import shutil

shutil.move('gtdata_lead_0.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_1.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_2.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_3.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_4.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_5.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_6.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_7.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_8.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_9.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_10.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('gtdata_lead_11.npy', "/content/drive/My Drive/Leadwise_data/")

'/content/drive/My Drive/Leadwise_data/gtdata_lead_11.npy'

In [ ]:
labels_lead0 = np.concatenate(lead_0_class_labels).astype(int)
labels_lead1 = np.concatenate(lead_1_class_labels).astype(int)
labels_lead2 = np.concatenate(lead_2_class_labels).astype(int)
labels_lead3 = np.concatenate(lead_3_class_labels).astype(int)
labels_lead4 = np.concatenate(lead_4_class_labels).astype(int)
labels_lead5 = np.concatenate(lead_5_class_labels).astype(int)
labels_lead6 = np.concatenate(lead_6_class_labels).astype(int)
labels_lead7 = np.concatenate(lead_7_class_labels).astype(int)
labels_lead8 = np.concatenate(lead_8_class_labels).astype(int)
labels_lead9 = np.concatenate(lead_9_class_labels).astype(int)
labels_lead10 = np.concatenate(lead_10_class_labels).astype(int)
labels_lead11 = np.concatenate(lead_11_class_labels).astype(int)

In [ ]:
np.save('labels_lead0',labels_lead0)
np.save('labels_lead1',labels_lead1)
np.save('labels_lead2',labels_lead2)
np.save('labels_lead3',labels_lead3)
np.save('labels_lead4',labels_lead4)
np.save('labels_lead5',labels_lead5)
np.save('labels_lead6',labels_lead6)
np.save('labels_lead7',labels_lead7)
np.save('labels_lead8',labels_lead8)
np.save('labels_lead9',labels_lead9)
np.save('labels_lead10',labels_lead10)
np.save('labels_lead11',labels_lead11)

In [ ]:
import shutil

shutil.move('labels_lead0.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead1.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead2.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead3.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead4.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead5.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead6.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead7.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead8.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead9.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead10.npy', "/content/drive/My Drive/Leadwise_data/")
shutil.move('labels_lead11.npy', "/content/drive/My Drive/Leadwise_data/")

'/content/drive/My Drive/Leadwise_data/labels_lead11.npy'

##Leadwise data acquisition

In [ ]:
import os
import pandas as pd
import numpy as np

leadwise_gt_path = "/content/drive/My Drive/Leadwise_data/gtdata_lead_4.npy"
labels_path = "/content/drive/My Drive/Leadwise_data/labels_lead4.npy"

In [ ]:
import numpy as np
from scipy.signal import resample_poly

gt_set = []
down_250set = []
down_125set = []
down_100set = []
down_50set = []
down_25set = []
down_20set = []
down_10set = []
down_5set = []
down_2set = []
down_1set = []

gt_data = np.load(leadwise_gt_path)
lead_output_labels = np.load(labels_path)
for entry in gt_data:
  gt_set.append(entry)
  down_250set.append(resample_poly(entry,up=250,down=500))
  down_125set.append(resample_poly(entry,up=125,down=500))
  down_100set.append(resample_poly(entry,up=100,down=500))
  down_50set.append(resample_poly(entry,up=50,down=500))
  down_25set.append(resample_poly(entry,up=25,down=500))
  down_20set.append(resample_poly(entry,up=20,down=500))
  down_10set.append(resample_poly(entry,up=10,down=500))
  down_5set.append(resample_poly(entry,up=5,down=500))
  down_2set.append(resample_poly(entry,up=2,down=500))
  down_1set.append(resample_poly(entry,up=1,down=500))

In [ ]:
np.save('down_250set_lead4',np.array(down_250set))

In [ ]:
import shutil

shutil.move('down_250set_lead4.npy', "/content/drive/My Drive/")

'/content/drive/My Drive/down_250set_lead4.npy'

In [ ]:
len(down_250set)

6877

In [ ]:
len(lead_output_labels[:4622])

4622